In [1]:
import numpy as np
import statsmodels.api as sm

from gmm import GELEstimator
np.random.seed(94305)

# Generalized Empirical Likelihood Estimation

We want to estimate a parameter implicitly defined as a solution to a moment condition $g(Z; \theta) = 0$. 

GMM proceeds by minimizing the quadratic form $Q(\theta) = g(Z; \theta)'Wg(Z; \theta)$, where $W$ is a positive definite weighting matrix. The choice of the weighting matrix is crucial for the efficiency of the estimator.

GEL sidesteps the choice of the weighting matrix by minimizing the empirical likelihood function. The empirical likelihood function is defined as the maximum of the likelihood function subject to the moment condition.

The (log) empirical likelihood function is defined as:

$$
L(\theta) = \max_{p \in \mathcal{P}} \sum_{i=1}^n \log p_i
$$

subject to the moment condition $$p_i g(Z; \theta) = 0$$
and adding up $\sum_{i=1}^n p_i = 1$.

where $\mathcal{P}$ is the set of all probability distributions on $\{1, \ldots, n\}$. At a first glance, this problem seems harder, since we need to solve for a weight vector $p$, which is typically of much higher dimension than the parameter $\theta$.


[Imbens, Spady, and Johnson (1998)](https://scholar.harvard.edu/sites/scholar.harvard.edu/files/imbens/files/information_theoretic_approaches_to_inference_in_moment_condition_models.pdf) show that minimizing the KLIC instead of the log empirical likelihood produces better behavior under `mild' misspecification. The corresponding Exponential Tilting estimator is defined as solving

$$
\max_{\theta, p_i} - \sum_{i=1}^n p_i \log p_i
$$

subject to the moment condition $p_i g(Z; \theta) = 0$ and $\sum_{i=1}^n p_i = 1$. 

The estimating equations are difficult to solve directly, so we instead use the saddlepoint representation

$$
\max_{\theta} \min_{t} \sum_{i=1}^n \exp(t' g(Z_i; \theta))
$$

that concentrates out the weight vector.
The inner function is strictly convex in $t$, and can be solved quickly. The outer function iterates over candidate values of $\theta$. We implement this approach in `GELEstimator`.


## Linear Problems

### estimating a mean

In [2]:
n = 1000
X = np.random.normal(np.array([1, 2]), size=(n, 2))
np.c_[X.mean(axis=0), np.sqrt(n/(n-1) * X.var(axis=0)/n)]

array([[1.01999056, 0.03015686],
       [2.04451165, 0.03196784]])

In [3]:
moment_cond_mean = lambda D, theta: D - theta

gelmod = GELEstimator(m = moment_cond_mean)
gelmod.fit(X, np.zeros(2))
gelmod.summary()

array([[1.01999054, 0.03015686],
       [2.04451167, 0.03196784]])

Analytic and GEL estimates identical.


### OLS

In [4]:
def dgp(n=100_000, beta=np.array([-0.5, 1.2]), rho=0.7, pi=np.array([0.5, -0.1])):
    ε = np.random.normal(0, 1, n)
    z = np.random.normal(0, 1, n * pi.shape[0]).reshape(n, pi.shape[0])
    # Generate endogenous x, influenced by the instrument
    x = z @ pi + ε * rho + np.random.normal(0, 1, n)
    X = np.c_[np.ones(n), x]
    # heteroskedasticity
    y = X @ beta + ε + np.random.normal(0, 1 + 2 * (X[:, 1] > 0) + 1 * (np.abs(X[:, 1]) > 0.5), n)
    return y, X, z

In [5]:
y, X, z = dgp(rho = 0)
print(sm.OLS(y, X).fit(cov_type = "HC2").summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4980      0.010    -51.823      0.000      -0.517      -0.479
x1             1.2048      0.009    130.295      0.000       1.187       1.223


__Estimation via EL/ET__

In [6]:
def moment_condition_ols(D, θ):
    y, X = D[:, 0], D[:, 1:]
    r = y - X @ θ
    return X * r[:, None]

D = np.c_[y, X]
gelmod = GELEstimator(m = moment_condition_ols, verbose=False)
k = X.shape[1]
gelmod.fit(D, np.zeros(k))
gelmod.summary()

array([[-0.49799059,  0.00961084],
       [ 1.20481178,  0.01167324]])

### IV

In [7]:
y, X, Z = dgp(rho = 1)
print(sm.OLS(y, X).fit(cov_type = "HC2").summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5031      0.010    -52.128      0.000      -0.522      -0.484
x1             1.6453      0.007    241.302      0.000       1.632       1.659


Biased. 

In [8]:
# 2SLS via control function
D = X[:, 1]
Dhat = sm.OLS(D, Z).fit().predict()
tslsreg = sm.OLS(y, np.c_[sm.add_constant(D), D - Dhat]).fit(cov_type="HC1")
tslsreg.params[1], tslsreg.bse[1]

(1.2129555645196368, 0.01892188485776335)

unbiased.

In [9]:
def moment_condition_iv(D, θ):
    y, X, Z = D[:, 0], D[:, 1:3], D[:, 3:]
    r = y - X @ θ
    return Z * r[:, None]


D = np.c_[y, X, Z]
k = X.shape[1]
gelmod = GELEstimator(m=moment_condition_iv)
gelmod.fit(D, np.zeros(k))
gelmod.summary()

array([[0.52258708, 0.01041044],
       [1.22171316, 0.01039752]])

Endogenous coefficient is correct

## Other Problems 

### Lin Regression

$$
Y_i = \alpha + \tau Z_i + X_i' \beta + Z_i \tilde{X}_i' \gamma + \epsilon_i
$$

where $\tilde{X}_i = X_i - \bar{X}$ is the centered version of $X_i$. 

In [10]:
n = 500
X = np.random.normal(0, 1, n * 2).reshape(n, 2)
Y0 = X[:, 0] + X[:, 0] ** 2 + np.random.uniform(-0.5, 0.5, n)
Y1 = X[:, 1] + X[:, 1] ** 2 + np.random.uniform(-1, 1, n)
Z = np.random.binomial(1, 0.6, n)
Y = Y0 * (1 - Z) + Y1 * Z
D = np.c_[Y, Z, X]

In [20]:
olslin = sm.OLS(
    Y, np.c_[sm.add_constant(Z), X, Z[:, None] * (X - X.mean(axis=0))]
).fit(cov_type = "HC2")
olslin.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
const,1.0189,0.098,10.413,0.000,0.827,1.211
x1,0.1907,0.139,1.371,0.170,-0.082,0.463
x2,0.9860,0.191,5.167,0.000,0.612,1.360
x3,0.1129,0.101,1.114,0.265,-0.086,0.311
x4,-0.8281,0.218,-3.794,0.000,-1.256,-0.400
x5,1.4133,0.213,6.633,0.000,0.996,1.831


The point estimates are consistent, but we do not propagate forward the uncertainty from estimating the sample mean $\bar{X}$. To do this, we could stack the moment conditions

$$
\begin{bmatrix}
X - \mu \\
[X, X - \mu] (y - [X, X - \mu] \beta)
\end{bmatrix}
$$

In [12]:
def moment_condition_lin(D, θ):
    Y, Z, X = D[:, 0], D[:, 1], D[:, 2:]
    n, p = X.shape
    mu, beta = θ[:p], θ[p:]
    Xcent = X - mu
    XX = np.c_[np.ones(n), Z, X, Z[:, None] * Xcent]
    r = XX * (Y - XX @ beta)[:, None]
    return np.c_[Xcent, r]
print(theta_init := np.r_[X.mean(axis=0), np.random.rand(6)])

[0.02681823 0.0657933  0.16969411 0.20839162 0.96615393 0.1186393
 0.76986621 0.31422366]


In [13]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [17]:
gelmod = GELEstimator(m=moment_condition_lin, min_method="COBYLA")
gelmod.fit(D, theta_init)

In [23]:
np.c_[gelmod.summary(), np.r_[X.mean(axis = 0), olslin.params]]

array([[ 0.02047514,  0.04419224,  0.02681823],
       [ 0.067958  ,  0.04785572,  0.0657933 ],
       [ 1.01598886,  0.07163751,  1.01894042],
       [ 0.20515589,  0.06153608,  0.1907456 ],
       [ 0.91805187,  0.0943588 ,  0.98603771],
       [ 0.11959629,  0.14415267,  0.1128657 ],
       [-0.74880967,  0.0628265 , -0.82811793],
       [ 1.42066068,  0.13805289,  1.41325515]])

This parameter vector has $\bar{X}$ as the first two elements, and the later are the regression coefficients. These look about right, although the standard errors are a bit too narrow?